In [1]:
using TaylorIntegration, ValidatedNumerics
using Plots, LaTeXStrings
# plotly()
pyplot(grid = :false, legend = :false)

Plots.PyPlotBackend()

In [2]:
const λ = 1.0
const Z = 2.0
const order = 28
const t0 = 0.0
const t_max = 10000.0 
const abs_tol = 1.0E-20
const steps = 500000

500000

In [3]:
function undrivenHelium(t, q)
    Q₁, P₁, Q₂, P₂ = q
    
    q₁ = 0.25*Q₂^2*P₁
    p₁ = 2Q₁*(-0.125P₂^2 + Z + Q₂^2/abs(Q₁^2 - Q₂^2) - Q₂^2) + Q₁^2*Q₂*(1/(Q₁^2 - Q₂^2)^2)*2Q₁
    q₂ = 0.25*Q₁^2*P₂
    p₂ = 2Q₂*(-0.125P₁^2 + Z + Q₁^2/abs(Q₁^2 - Q₂^2) - Q₁^2) + Q₁^2*Q₂*(1/(Q₁^2 - Q₂^2)^2)*2Q₂
    
    return [q₁, p₁, q₂, p₂] 

end

undrivenHelium (generic function with 1 method)

The Taylor integration:

In [ ]:
?taylorinteg

In [4]:
function condini(x10::Float64, px10::Float64)
    @assert x10 != 0
    Q1 = sqrt(x10)
    Q2 = 0.0
    P1 = px10*sqrt(x10)    
    P2 = sqrt(8Z)    
    return [Q1, P1, Q2, P2]
end

condini (generic function with 1 method)

In [5]:
q0 = condini(2.1, 1.5)

4-element Array{Float64,1}:
 1.44914
 2.17371
 0.0    
 4.0    

In [6]:
t, q = taylorinteg(undrivenHelium, q0, t0, 0.01, order, abs_tol, maxsteps=2); #warm-up lap
@time t, q = taylorinteg(undrivenHelium, q0, t0, t_max, order, abs_tol, maxsteps=steps);

149.649714 seconds (2.26 G allocations: 209.731 GB, 17.66% gc time)


The final state:

In [12]:
function regHam(Q₁, P₁, Q₂, P₂)
    #Cantidades auxiliares
    P₁² = P₁^2
    P₂² = P₂^2
    Q₁² = Q₁^2
    Q₂² = Q₂^2
    nf = abs((Q₁)^2 - (Q₂)^2)
    
    H = 0.125*(P₁²*Q₂² + P₂²*Q₁²) - Z*(Q₁² + Q₂²) + Q₁²*Q₂²*(1.0 + 1.0/nf)
    return H
end


regHam(v) = regHam(v...)

regHam (generic function with 2 methods)

In [7]:
t[end], q[end,:]

(0.6671444308387554,[3.32788,6.61075e8,3.32788,6.61075e8])

In [13]:
regHam(q[end,:])

1.2099799097401684e18